# B"H

In [1]:
# SEE: 
#    - https://cloud.google.com/bigquery/docs/reference/libraries
#    - https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html

# Imports the Google Cloud client library
from google.cloud import bigquery


In [4]:
%set_env GOOGLE_APPLICATION_CREDENTIALS=/home/laz/app-keys/stats-learning-d392db36f6a4.json

env: GOOGLE_APPLICATION_CREDENTIALS=/home/laz/app-keys/stats-learning-d392db36f6a4.json


In [5]:

# Instantiates a client
bq_client = bigquery.Client(project='stats-learning')

In [6]:
bq_client

---

## List datasets for the client’s project:

In [7]:
datasets = list(bq_client.list_datasets())
project = bq_client.project

if datasets:
    print('Datasets in project {}:'.format(project))
    for dataset in datasets:  # API request(s)
        print('\t{}'.format(dataset.dataset_id))
else:
    print('{} project does not contain any datasets.'.format(project))

Datasets in project stats-learning:
	ds_stats
	pinpoint


## List tables for the dataset:

In [9]:
dataset_ref = bq_client.dataset('pinpoint')

tables = list(bq_client.list_tables(dataset_ref))  # API request(s)

In [10]:
tables

---
## Extract a table to Google Cloud Storage:

In [11]:
bucket_name     = 'bucket-stats'

destination_uri = 'gs://{}/{}'.format(bucket_name, 'test-export.csv')

# -- ------------------------------------------------------------------

dataset_ref     = bq_client.dataset('pinpoint')

table_ref       = dataset_ref.table('x_stg_CREATE_TEST')

# -- ------------------------------------------------------------------

extract_job = bq_client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location='US'
)  

extract_job.result()  # Waits for job to complete.

---
## Overwrite / replace an existing table with a CSV file from Cloud Storage:

In [12]:
# -- ------------------------------------------------------------------
table_ref = dataset_ref.table('x_stg_CREATE_TEST')
# -- ------------------------------------------------------------------



# -- ------------------------------------------------------------------
job_config = bigquery.LoadJobConfig()

job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job_config.skip_leading_rows = 1

# The source format defaults to CSV, so the line below is optional.
job_config.source_format = bigquery.SourceFormat.CSV
# -- ------------------------------------------------------------------



# -- ------------------------------------------------------------------
load_job = bq_client.load_table_from_uri(    
    'gs://bucket-stats/test-export.csv',
    table_ref,
    job_config = job_config
)  # API request

assert load_job.job_type == 'load'

load_job.result()  # Waits for table load to complete.
# -- ------------------------------------------------------------------



# -- ------------------------------------------------------------------
assert load_job.state == 'DONE'
assert bq_client.get_table(table_ref).num_rows == 3
# -- ------------------------------------------------------------------

---
## Writing query results to a destination table:

In [13]:
# -- -----------------------------------------------------
job_config = bigquery.QueryJobConfig()

# Set the destination table. 
table_ref = dataset_ref.table('x_stg_create_from_query')

job_config.destination = table_ref

# The write_disposition specifies the behavior when writing query results
# to a table that already exists. With WRITE_TRUNCATE, any existing rows
# in the table are overwritten by the query results.
job_config.write_disposition = 'WRITE_TRUNCATE'
# -- -----------------------------------------------------


# -- -----------------------------------------------------
# Start the query, passing in the extra configuration.
query_str = (
    'SELECT   my_dates, my_int64 '
    'FROM     `stats-learning.pinpoint.x_stg_CREATE_TEST`  '
    'WHERE    my_int64 > 1 '
)

query_job = bq_client.query(
    query_str,
    # Location must match that of the dataset(s) referenced in the query and of the destination table.
    location   = 'US',
    job_config = job_config
)  # API request - starts the query

rows = list(query_job)  # Waits for the query to finish

assert len(rows) == 2
# -- -----------------------------------------------------

In [14]:
len(rows)

2